In [1]:
import selenium
from selenium import webdriver
import time 
import re
import csv
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import math
import json




In [3]:
zips = pd.read_csv('./smh_zips.txt')

In [9]:
zip_codes = zips['ZipCode']
zip_codes =zip_codes.tolist()

In [9]:
driver = webdriver.Chrome(r'C:\Users\jackc\chromedriver_win32\chromedriver.exe')

zips_skipped = [] 
## Using two random zip codes for now

## Make an empty list to store dictionaries later 
list_of_dicts = []

zip_codes = [20852]
for element in zip_codes:

    print("Starting on " , element)

    try:
        ## Navigate to the link based on the zipcodes provided in the list 'zips'
        link ='https://www.greatschools.org/search/search.zipcode?sort=rating&zip=' + str(element) 
        driver.get(link)

        ## Find the total number of pages per zip based on total number of schools 
        num_pages = driver.find_element_by_xpath('.//div[@class="pagination-summary"]').text
        total_schools = int(num_pages[19:21])
        num_pages = int(math.ceil(total_schools/25))

        ## For each page in each zip, loop through and pull out the relevant information
        for x in range(num_pages):
            page_num = x + 1
            if(page_num > 1):
                link = 'https://www.greatschools.org/search/search.zipcode?page=' + str(page_num) + '&sort=rating&view=list&zip='+str(element)
                driver.get(link)


            ## get a list of all of the schools on the left 
            schools = driver.find_elements_by_xpath('//li[@class=" unsaved"]')

            ## Loop through all schools 
            for school in schools: 
                name = school.find_element_by_xpath('.//a[@class="name"]').text
                address = school.find_element_by_xpath('.//div[@class="address"]').text

                ## Find level and enrollment -- they share the same tag, so separate them out
                ## Note, sometimes enrollment is not available, so check if it is and if not, make it na

                level_and_enrollment = school.find_elements_by_xpath('.//span[@class="open-sans_sb"]')
                if(len(level_and_enrollment) > 1):
                    level = level_and_enrollment[0].text
                    enrollment = level_and_enrollment[1].text
                else:
                    level = level_and_enrollment[0].text
                    enrollment = 'NA'

                rating = school.find_elements_by_xpath('.//span[@class="tipso_style"]')
                rating = rating[0].text[0]

                #If rating is unavaiable, will return 'C' -- change that to 'NA'
                if(rating == 'C'):
                    rating = 'NA'

                scale = school.find_elements_by_xpath('.//div[@class="scale"]')
                scale = scale[0].text

                ## make a dictionary for each entry and add it to the list_of_dicts
                dict = {}
                dict['name'] = name
                dict['address'] = address
                dict['level'] = level
                dict['enrollment'] = enrollment
                dict['rating'] = rating
                dict['scale'] = scale
                dict['zip'] = element
                list_of_dicts.append(dict)

        print('Finished' , element )
    except:
        print('skipped ', element)
        zips_skipped.append(element)
        # next = driver.find_elements_by_xpath('.//a[@class="anchor-button   anchor-button"]')
        # next[1].click() 

Starting on  20852
Finished 20852


In [10]:
final_df = pd.DataFrame(list_of_dicts)

In [11]:
dict = list_of_dicts[0]

In [19]:
def write_json(new_data, filename='scraped_school_data.json'):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with files_data inside emp_details
        file_data["emp_details"].append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)
 
